In [ ]:
import pandas as pd
from bertopic import BERTopic
import json
import openai
from transformers import pipeline

In [3]:
# load saved BERTopic Model, generate topics based on random 100k sample of the BostonGlobe (lexisnexis) data
topic_model = BERTopic.load("/Users/mvoong/Downloads/bglobe_519_body") # for the scc: /projectnb/sparkgrp/mvoong/bglobe_519_body

In [4]:
# topic -1 are all outliers
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,67888,-1_iraq_selectmen_property_child
1,0,14321,0_ooops_wham_downtime_blah
2,1,4612,1_inning_yankees_homer_francona
3,2,3412,2_brady_parcells_dolphins_kraft
4,3,2885,3_celtics_ainge_rondo_rivers
...,...,...,...
942,941,10,941_bluestone_amoxicillin_relman_hpv
943,942,10,942_zoltan_elizabet_quints_retinas
944,943,10,943_masstlc_hopcroft_retraining_carbonite
945,944,10,944_yen_deflation_trichet_storchak


In [4]:
# see the articles
bglobe_sample = pd.read_csv("/Users/mvoong/Downloads/bglobe_100k_sample.csv") # for the scc: /projectnb/sparkgrp/mvoong/bglobe_100k_sample.csv
bglobe_sample.head(10)

,Unnamed: 0,pub_type,position_section,position_subsection,hl1,hl2,author,lede,body,language,word_count,copyright,content-id,volume,issue_number,edition,pub_name,pub_date,licensor_indexing_terms,indexing_terms
0,12862,NaN,METRO/REGION; Pg. B2,NaN,MWRA issues 1st water report;,NaN,NaN,The Massachusetts Water Resources Authority ha...,"Locally, just about any level of reporting wou...",['ENGLISH'],326.0,Copyright 1996 Globe Newspaper Company,NaN,NaN,NaN,NaN,The Boston Globe,"{'month': '08', 'day': '25', 'year': '1996'}",NaN,{'legal': [{'className': 'Business & Corporate...
1,17795,"Newspaper, Newspapers",EDITORIAL OPINION,Opinion,NaN,NaN,NaN,OBAMA MIDEAST SPEECH ``I guess I'm a little go...,- JEFFREY GOLDBERG www.theatlantic.com ``Presi...,['ENGLISH'],363.0,Copyright 2011 Globe Newspaper Company All Rig...,BGLOBE-G7C1PGAVO.3,NaN,NaN,NaN,The Boston Globe,"{'day': '24', 'month': '05', 'year': '2011'}",NaN,"{'subject': [{'score': '90', 'classCode': 'ST0..."
2,1036,NaN,METRO/REGION; Pg. B3,NaN,Curley jury process begins in private;,NaN,NaN,Judge Judith A. Cowin yesterday barred report...,CAMBRIDGE - Jury selection began behind clos...,['ENGLISH'],331.0,Copyright 1998 Globe Newspaper Company,NaN,NaN,NaN,",City Edition",The Boston Globe,"{'month': '10', 'day': '21', 'year': '1998'}",NaN,"{'subject': [{'score': '90', 'classCode': 'STX..."
3,13596,NaN,METRO/REGION; Pg. 28,NaN,Pols look for that union label;,NaN,NaN,"Touting their connections to labor unions, Cit...","""I want to be the point man for labor to go to...",['ENGLISH'],614.0,Copyright 1995 Globe Newspaper Company,NaN,NaN,NaN,NaN,The Boston Globe,"{'month': '09', 'day': '05', 'year': '1995'}",NaN,"{'legal': [{'className': 'Tax Law', 'classCode..."
4,1669,NaN,CITY WEEKLY; Pg. 1,NaN,Hillary Clinton touts Somerville as a can-do c...,NaN,NaN,Elected officials and regular folks warmly we...,SOMERVILLE - Hillary Rodham Clinton's visit ...,['ENGLISH'],842.0,Copyright 1998 Globe Newspaper Company,NaN,NaN,NaN,",City Edition",The Boston Globe,"{'month': '10', 'day': '25', 'year': '1998'}",NaN,"{'legal': [{'className': 'Education Law', 'cla..."
5,10466,NaN,EDITORIAL PAGE; Pg. A26,NaN,Remembering Malcolm X;,NaN,NaN,"At the height of Newt Gingrich's popularity, I...",This thought seems to have come from history's...,['ENGLISH'],330.0,Copyright 1997 Globe Newspaper Company,NaN,NaN,NaN,NaN,The Boston Globe,"{'month': '06', 'day': '06', 'year': '1997'}",NaN,"{'subject': [{'score': '77', 'classCode': 'ST0..."
6,6011,NaN,METRO/REGION; Pg. B14,NaN,School bus crash linked to drinking,NaN,NaN,Franklin Carrier told police he hadn't been d...,"LACONIA, N.H. - A man who crashed his car in...",['ENGLISH'],226.0,Copyright 1998 Globe Newspaper Company,NaN,NaN,NaN,",City Edition",The Boston Globe,"{'month': '11', 'day': '22', 'year': '1998'}",NaN,"{'subject': [{'score': '90', 'classCode': 'ST0..."
7,19292,"Newspaper, Newspapers",SPORTS,NaN,"Red Sox 7, Indians 4",NaN,NaN,at Fenway Park CLEVELAND AB R H BI BB SO Avg. ...,Aviles 2b 5 1 1 0 0 0 .276 Cabrera ss 5 1 3 1 ...,['ENGLISH'],267.0,Copyright 2013 Globe Newspaper Company All Rig...,BGLOBE-58872e78-c57f-11e2-9f23-d6f63dfce94a,NaN,NaN,NaN,The Boston Globe,"{'day': '26', 'month': '05', 'year': '2013'}",NaN,"{'city': [{'score': '90', 'classCode': 'GX567'..."
8,17069,"Newspaper, Newspapers",SPORTS,NaN,Two late picks earn Titans crucial victory,NaN,NaN,Randy Bullock made a 44-yard field goal with 4...,BODY Wentz shook it off quickly. On the ensuin...,['ENGLISH'],1266.0,Copyright 2021 Globe Newspaper Company All Rig...,BGLOBE-b0cc609e3aa611ec91b5eae6a5365c28,NaN,NaN,NaN,The Boston Globe,"{'day': '01', 'month': '11', 'year': '2021'}",NaN,{'legal': [{'className': 'Business & Corporate...
9,12189,NaN,ECONOMY; Pg. C1,NaN,Divided loyalties;,NaN,NaN,"To bankers, it's strictly business. But to oth...","""What we have done for the state of Rhode Isla...",['ENGLISH'],684.0,Copyright 1997 Globe Newspaper Company,NaN,NaN,NaN,NaN,The Boston Globe,"{'month': '09', 'day': 

In [5]:
# get docs for bert topic model - article body 
body = bglobe_sample['body']
topic_model.get_document_info(body)[:5]

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,"Locally, just about any level of reporting wou...",24,24_mwra_sewage_dam_devillars,mwra - sewage - dam - devillars - pfas - pcbs ...,1.0,False
1,- JEFFREY GOLDBERG www.theatlantic.com ``Presi...,-1,-1_iraq_selectmen_property_child,iraq - selectmen - property - child - mayor - ...,0.0,False
2,CAMBRIDGE - Jury selection began behind clos...,-1,-1_iraq_selectmen_property_child,iraq - selectmen - property - child - mayor - ...,0.0,False
3,"""I want to be the point man for labor to go to...",-1,-1_iraq_selectmen_property_child,iraq - selectmen - property - child - mayor - ...,0.0,False
4,SOMERVILLE - Hillary Rodham Clinton's visit ...,-1,-1_iraq_selectmen_property_child,iraq - selectmen - property - child - mayor - ...,0.0,False


In [6]:
# see bag of words for topic 1 
topic_model.get_topic(1)

[('inning', 0.11658965295023382),
 ('yankees', 0.10596102502492137),
 ('homer', 0.10563546411665027),
 ('francona', 0.1016020351007049),
 ('hits', 0.09718864049137246),
 ('pitched', 0.09581969205055282),
 ('cora', 0.09569339427567942),
 ('fastball', 0.09522087264278532),
 ('righthander', 0.09321927524403119),
 ('martinez', 0.09154655818859768)]

In [92]:
def get_relevant_indexing_terms(sample):
    # cleaning sample data, add topic column  
    # get list of possible topic labels from the indexing terms (lexisnexis labels)
    # there's probably a much better way of doing this...
    # to do: also if there is more than one label - they should be ranked for evaluating openai label   
    possible_topics = []
    for terms in sample['indexing_terms']:
        temp = []
        try:
            terms = eval(terms)
            if terms['subject']:
                for v in terms['subject']:
                    if int(v['score']) > 85:
                        temp.append(v['className'])
            elif terms['industry']:
                for v in terms['industry']:
                    if int(v['industry']['score']) > 85:
                        temp.append(v['className'])
            elif terms['legal']:
                for v in terms['legal']:
                    temp.append(v['className'])
            possible_topics.append(temp)
        except KeyError:
            # no subjects, industry, or legal at all or above 85 
            possible_topics.append("") 
        except TypeError:
            # no indexing_terms 
            possible_topics.append("")
    return possible_topics

In [93]:
# add possible topics as new column in dataframe
bglobe_sample['possible_topics'] = get_relevant_indexing_terms(bglobe_sample)
bglobe_sample.head()

,Unnamed: 0,pub_type,position_section,position_subsection,hl1,hl2,author,lede,body,language,...,volume,issue_number,edition,pub_name,pub_date,licensor_indexing_terms,indexing_terms,possible_topics,bertopic_topic_label,openai_label
0,12862,NaN,METRO/REGION; Pg. B2,NaN,MWRA issues 1st water report;,NaN,NaN,The Massachusetts Water Resources Authority ha...,"Locally, just about any level of reporting wou...",['ENGLISH'],...,NaN,NaN,NaN,The Boston Globe,"{'month': '08', 'day': '25', 'year': '1996'}",NaN,{'legal': [{'className': 'Business & Corporate...,"[BACTERIA, COLIFORM BACTERIA, WATER QUALITY, E...",24,Water Contamination
1,17795,"Newspaper, Newspapers",EDITORIAL OPINION,Opinion,NaN,NaN,NaN,OBAMA MIDEAST SPEECH ``I guess I'm a little go...,- JEFFREY GOLDBERG www.theatlantic.com ``Presi...,['ENGLISH'],...,NaN,NaN,NaN,The Boston Globe,"{'day': '24', 'month': '05', 'year': '2011'}",NaN,"{'subject': [{'score': '90', 'classCode': 'ST0...","[US REPUBLICAN PARTY, US PRESIDENTIAL CANDIDAT...",-1,
2,1036,NaN,METRO/REGION; Pg. B3,NaN,Curley jury process begins in private;,NaN,NaN,Judge Judith A. Cowin yesterday barred report...,CAMBRIDGE - Jury selection began behind clos...,['ENGLISH'],...,NaN,NaN,",City Edition",The Boston Globe,"{'month': '10', 'day': '21', 'year': '1998'}",NaN,"{'subject': [{'score': '90', 'classCode': 'STX...","[JURY TRIALS, JURY DUTY, LAWYERS, PETITIONS]",-1,
3,13596,NaN,METRO/REGION; Pg. 28,NaN,Pols look for that union label;,NaN,NaN,"Touting their connections to labor unions, Cit...","""I want to be the point man for labor to go to...",['ENGLISH'],...,NaN,NaN,NaN,The Boston Globe,"{'month': '09', 'day': '05', 'year': '1995'}",NaN,"{'legal': [{'className': 'Tax Law', 'classCode...","[LABOR UNIONS, CITY GOVERNMENT, CITIES, Local ...",-1,
4,1669,NaN,CITY WEEKLY; Pg. 1,NaN,Hillary Clinton touts Somerville as a can-do c...,NaN,NaN,Elected officials and regular folks warmly we...,SOMERVILLE - Hillary Rodham Clinton's visit ...,['ENGLISH'],...,NaN,NaN,",City Edition",The Boston Globe,"{'month': '10', 'day': '25', 'year': '1998'}",NaN,"{'legal': [{'className': 'Education Law', 'cla...","[CITY GOVERNMENT, EDUCATION SYSTEMS & INSTITUT...",-1,


In [10]:
# get the bag of words (keywords) for each topic from the bertopic model
topics_df = topic_model.get_topic_info()
word_bags = [""]
for i in range(len(topics_df)):
    temp = topic_model.get_topic(i)
    try:
        words = [x for x,_ in temp]
        word_bags.append(words)
    except TypeError:
        pass

topics_df["Keywords"] = word_bags

In [16]:
# use openai api to create a label for the bag of words from bertopic
openai.api_key = ""

# exponential back off - because i kept getting ratelimiterror 
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [ ]:
# get labels for all bag of words from bertopic
for i in range(len(topics_df)):
    labels = topics_df['Keywords'][i]
    
    # build prompt for openai 
    formatted_keywords = ", ".join(labels[:-1]) + ", and " + labels[-1] + ".\n"
    text_input = "Please summarize this list of words into one term: " + formatted_keywords
    text_input += "Please generalize the final term.\n"

    # using gpt-3.5-turbo: $0.002/1,000 tokens
    # each request is ~150 tokens (including the response from openai)
    background_prompt = [{"role": "system", "content": "You are summarizing a list of words. Be as concise and informative as possible."},
                         {"role": "user", "content": "Please summarize this list of words into one term: covid, deaths, confirmed, baker, average, numbers, hospitals, dph, weeks, clusters. Please generalize the final term."},
                         {"role": "assistant", "content": "Public Health"}]


    prompt = background_prompt + [{"role": "user", "content": text_input}]

    response = completion_with_backoff(model="gpt-3.5-turbo",
                                       temperature=1, 
                                       max_tokens=5,
                                       messages=prompt)

    # save result in json file
    with open('open_ai_label.json', 'r+') as f:
        # load existing data 
        file_data = json.load(f)
        file_data[int(topics_df['Topic'][i])] = {"Name": topics_df['Name'][i],
                                              "OpenAI_label": response['choices'][0]['message']['content'],
                                              "OpenAI_metadata": response}
    
        f.seek(0)
        # convert back to json
        json.dump(file_data, f, indent=4)

In [39]:
# combine articles with topic labels from open ai 
f = open('open_ai_label.json')
openai_labels = json.load(f)

# add topic number from bertopic to bglobe dataframe in new column
bglobe_sample['bertopic_topic_label'] = topic_model.get_document_info(body)['Topic']

# add open ai label to bglobe dataframe in new column    
label_name = [openai_labels[str(bglobe_sample['bertopic_topic_label'][i])]['OpenAI_label'] 
              if int(bglobe_sample['bertopic_topic_label'][i]) != -1 else "" for i in range(len(bglobe_sample))]
bglobe_sample['openai_label'] = label_name


In [113]:
# use boston_herald_test.csv to assign bertopic labels and test the openai labels 
# get the relevant lexisnexis indexing terms
test_articles = pd.read_csv("boston_herald_test.csv")
test_articles['possible_topics'] = get_relevant_indexing_terms(test_articles)

In [114]:
# replace nan body with ""
for i in range(len(test_articles)):
    if type(test_articles['body'][i]) != str:
        test_articles['body'][i] = ""


In [130]:
# get bertopics for each article
topics, probs = topic_model.transform(test_articles['body'])
test_articles['bertopic_topic_label'] = topics

# add open ai label to bglobe dataframe in new column    
test_label_name = [openai_labels[str(test_articles['bertopic_topic_label'][i])]['OpenAI_label'] 
              if int(test_articles['bertopic_topic_label'][i]) != -1 else "" for i in range(len(test_articles))]
test_articles['openai_label'] = test_label_name

In [132]:
test_articles

,pub_type,position_section,position_subsection,hl1,hl2,author,lede,body,language,word_count,...,issue_number,edition,pub_name,pub_date,licensor_indexing_terms,indexing_terms,possible_topics,bertopic_topic_label\t,bertopic_topic_label,openai_label
0,"Newspaper, Newspapers",NEWS,Local,3-alarm blaze leaves some homeless,NaN,COLNETH SMILEY Jr.,A three-alarm blaze at a Quincy apartment buil...,Smoke from the fire on Newport Street across f...,['ENGLISH'],158,...,No. 309,All Editions,The Boston Herald,5/11/15,"FIRE, NEWPORT, QUINCY, ALARM, SMOKE, SMYTH, 30...","{'legal': [{'className': 'Governments', 'class...",[FIRES],16,16,Fire Safety
1,"Newspaper, Newspapers",NEWS,Local,Bridgewater police seek ID of burned woman's body,NaN,BRIAN DOWLING,Police are asking for the public's help to ide...,"The remains of the unidentified woman, whose h...",['ENGLISH'],201,...,No. 309,All Editions,The Boston Herald,5/11/15,"WOMAN, POLICE, BODY, BRIDGEWATER, BURNED, CRUZ...","{'subject': [{'score': '90', 'classCode': 'N92...",[POLICE FORCES],-1,-1,
2,"Newspaper, Newspapers",SPORTS,Football-NFL,Fitzpatrick gets thumbs-up to start,NaN,NaN,It's a thumbs-up on Ryan Fitzpatrick starting ...,"'No limitations today,' Fitzpatrick said. 'Rea...",['ENGLISH'],514,...,No. 309,All Editions,The Boston Herald,5/11/15,"FITZPATRICK, MANZIEL, COACH, FISHER, QUARTERBA...","{'legal': [{'className': 'Education Law', 'cla...","[ATHLETES, AMERICAN FOOTBALL]",-1,2,Football.
3,"Newspaper, Newspapers",NEWS,Obituaries,"Eileen Walsh, at 83, - of Abington, a nurse",NaN,NaN,"Eileen (Conlon) Walsh of Abington, a nurse, di...",She was a nurse at Shattuck and Carney hospita...,['ENGLISH'],111,...,No. 309,All Editions,The Boston Herald,5/11/15,"NURSE, WALSH, ABINGTON, EILEEN, FUNERAL, ARRAN...","{'subject': [{'score': '97', 'classCode': 'STX...",[DEATHS & OBITUARIES],4,4,Personal Life/Obit
4,"Newspaper, Newspapers",SPORTS,Agate,HIGH SCHOOL SCOREBOARD,NaN,NaN,"FIELD HOCKEY Governor's 5, Groton 1 St. Mark's...","Newton North at Needham, 4 Chelmsford at North...",['ENGLISH'],937,...,No. 309,All Editions,The Boston Herald,5/11/15,"BISHOP, CATHOLIC, CENTRAL, DIVISION, NORTH, PR...","{'subject': [{'score': '89', 'classCode': 'ST0...",[CATHOLICS & CATHOLICISM],436,436,Prep Schools.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,"Newspaper, Newspapers",NEWS,Local,Women make history in Hub and the state,NaN,Joyce Ferriabough Bolling,The post-election story in the media was how t...,But there was another important story: Boston'...,['ENGLISH'],444,...,No. 309,All Editions,The Boston Herald,5/11/15,"WOMEN, ELECTED, COLOR, COUNCIL, WOMAN, BOSTON,...","{'legal': [{'className': 'Governments', 'class...","[CITY GOVERNMENT, CITIES, WOMEN, Local Governm...",-1,-1,
88,"Newspaper, Newspapers",OPINION,Editorials,A mayor in reverse,NaN,NaN,Boston Mayor Marty Walsh has learned the hard ...,"Walsh, the city's top cheerleader for a planne...",['ENGLISH'],337,...,No. 309,All Editions,The Boston Herald,5/11/15,"AGREEMENT, BOSTON, EDITOR, EVENT, LEARNED, MAY...","{'legal': [{'className': 'Governments', 'class...","[AGREEMENTS, CITY GOVERNMENT, Mayors]",137,137,Olympic Hosting Bid
89,"Newspaper, Newspapers",NEWS,Local,Fentanyl OD data lagging - in Bay State,NaN,MATT STOUT,As the tide of fentanyl grows in Massachusetts...,The information gap comes as federal officials...,['ENGLISH'],491,...,No. 309,All Editions,The Boston Herald,5/11/15,"FENTANYL, OFFICIALS, STATE, RELATED, DATA, HEA...","{'subject': [{'score': '89', 'classCode': 'ST0...","[HEALTH DEPARTMENTS, HEROIN, PUBLIC HEALTH ADM...",55,55,Opioid Crisis
90,"Newspaper, Newspapers",SPORTS,Basketball-Celtics,Slow start angers Celts,In need of quick fix after falling to 1-3,STEVE BULPETT,INDIANAPOLIS - The Celtics have dropped three ...,'Something's got to change. I don't know what ...,['ENGLISH'],899,...,No. 309,All Editions,The Boston Herald,5/11/15,"CELTICS, CELTS, BRADLEY, DIDN, GAMES, PACERS, ...",{'legal': [{'cl

In [133]:
# remove articles with bertopic label -1 - these are outliers that don't have a label 
# to do: look into if it is possible to use bertopic to assign a label to the outliers anyways 

# for now - we will look at articles that have a label other than -1 
test_articles_labeled = test_articles[test_articles['bertopic_topic_label'] != -1]

# evaluate the similarity between words from lexis nexis and openai label 
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sequence_to_classify = test_articles_labeled['openai_label'][0]
candidate_labels = test_articles_labeled['possible_topics'][0]
classifier(sequence_to_classify, candidate_labels, multi_class=True)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


{'sequence': 'Fire Safety',
 'labels': ['FIRES'],
 'scores': [0.8304260969161987]}

In [134]:
# attempt at evaluation using bart-mnli zero shot classification 
# get the max score from the candidate labels (lexisnexis labels) compared to the openai label
scores = []
for id, row in test_articles.iterrows():
    try:
        sequence_to_classify = row['openai_label']
        candidate_labels = row['possible_topics']
        result = classifier(sequence_to_classify, candidate_labels, multi_label=True)
        max_score = max(result['scores'])
        scores.append(max_score)
    except ValueError:
        scores.append("")

In [137]:
# add the scores to the test_articles dataframe
test_articles['eval_score'] = scores

In [139]:
test_articles.to_csv("eval_output.csv")

In [ ]:
# to do - lower threshold for indexing terms if there are none that are above 85 